# PROJECT: Early crop disease detection

# Preliminaries

In [1]:
!nvidia-smi # to see what GPU you have

Thu Dec  8 14:21:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   20C    P8    13W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install wandb --quiet

In [3]:
!pip install torchvision

In [1]:
import torch
from torchsummary import summary
import torchvision #This library is used for image-based operations (Augmentations)
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms, models  # datsets  , transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
# from warmup_scheduler import GradualWarmupScheduler
import glob
import wandb
from numba import cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [5]:
# from google.colab import drive # Link your drive if you are a colab user
# drive.mount('/content/drive') # Models in this HW take a long time to get trained and make sure to save it her

# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data from Mendley

In [2]:
# !mkdir './content/'
# !mkdir './content/data'
# # importing libraries
import requests, zipfile, io

"""
	Download images folder from given url, move it to dataset folder.
"""

#url for data without augmentation
url = "https://data.mendeley.com/datasets/tywbtsjrjv/1/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/Plant_leaf_diseases_dataset_without_augmentation.zip?dl=1"

# # url for data with augmentation
# url = "https://data.mendeley.com/datasets/tywbtsjrjv/1/files/b4e3a32f-c0bd-4060-81e9-6144231f2520/Plant_leaf_diseases_dataset_with_augmentation.zip?dl=1"
response = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall()

# Configs

In [3]:
config = {
    'batch_size': 4, # Increase this if your GPU can handle it
    'lr': 0.001,
    'checkpointPath' : ['checkpoit_CONVNEXT.pth','checkpoint_RESNET.pth'],
    'epochs': 10, # 10 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
}

# Classification Dataset

In [4]:
# DATA_DIR = '/content/Plant_leave_diseases_dataset_with_augmentation'# TODO: Path where you have downloaded the data
# # TRAIN_DIR = os.path.join(DATA_DIR, "classification/train") 
# # VAL_DIR = os.path.join(DATA_DIR, "classification/dev")
# # TEST_DIR = os.path.join(DATA_DIR, "classification/test")



# # # dataset = CustomDatasetFromCSV(my_path)
# # batch_size = 64
# # validation_split = .2
# # shuffle_dataset = True
# # random_seed= 42

# # # Creating data indices for training and validation splits:
# # dataset_size = len(dataset)
# # indices = list(range(dataset_size))
# # split = int(np.floor(validation_split * dataset_size))
# # if shuffle_dataset :
# #     np.random.seed(random_seed)
# #     np.random.shuffle(indices)
# # train_indices, val_indices = indices[split:], indices[:split]

# # # Creating PT data samplers and loaders:
# # train_sampler = SubsetRandomSampler(train_indices)
# # valid_sampler = SubsetRandomSampler(val_indices)

# # train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
# #                                            sampler=train_sampler)
# # validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
# #                                                 sampler=valid_sampler)

# # Transforms using torchvision - Refer https://pytorch.org/vision/stable/transforms.html

# # train_transforms = torchvision.transforms.Compose([ 
# #     # Implementing the right transforms/augmentation methods is key to improving performance.
# #                     torchvision.transforms.RandAugment(2,14),
# #                     torchvision.transforms.RandomHorizontalFlip(p=0.5),
# # #                     torchvision.transforms.RandomResizedCrop(size=(224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
# #                     torchvision.transforms.ToTensor(),
# #                     torchvision.transforms.Normalize(mean=0, std=1, inplace=False)
# #                     ])

# transform = transforms.Compose(
#     [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
# )
# # Most torchvision transforms are done on PIL images. So you convert it into a tensor at the end with ToTensor()
# # But there are some transforms which are performed after ToTensor() : e.g - Normalization
# # Normalization Tip - Do not blindly use normalization that is not suitable for this dataset

# # val_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])


# # train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR, transform = train_transforms)
# # val_dataset = torchvision.datasets.ImageFolder(VAL_DIR, transform = val_transforms)
# # # You should NOT have data augmentation on the validation set. Why?

# dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

# indices = list(range(len(dataset)))
# split = int(np.floor(0.85 * len(dataset)))  # train_size
# validation = int(np.floor(0.70 * split))  # validation
# print(0, validation, split, len(dataset))

# train_indices, validation_indices, test_indices = (
#     indices[:validation],
#     indices[validation:split],
#     indices[split:],
# )

# train_sampler = SubsetRandomSampler(train_indices)
# val_sampler = SubsetRandomSampler(validation_indices)
# test_sampler = SubsetRandomSampler(test_indices)

# targets_size = len(dataset.class_to_idx)
# # Create data loaders
# # train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = config['batch_size'], 
# #                                            shuffle = True,num_workers = 4, pin_memory = True)
# # val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = config['batch_size'], 
# #                                          shuffle = False, num_workers = 2)

# batch_size = 64
# train_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=batch_size, sampler=train_sampler
# )
# test_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=batch_size, sampler=test_sampler
# )
# val_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=batch_size, sampler=val_sampler
# )

In [5]:
transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'Plant_leave_diseases_dataset_without_augmentation'
image_datasets = {x: datasets.ImageFolder(data_dir, transform=transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}
data_size = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# You can do this with ImageFolder as well, but it requires some tweaking
class ClassificationTestDataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transforms):
        self.data_dir   = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in the test directory
        self.img_paths  = list(map(lambda fname: os.path.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [7]:
# Testing the test_dataset and test_loader
# test_dataset = ClassificationTestDataset(TEST_DIR, transforms = val_transforms) #Why are we using val_transforms for Test Data?
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = config['batch_size'], shuffle = False,
#                          drop_last = False, num_workers = 2)

In [8]:
# print("Number of classes: ", len(dataset.classes))
# print("No. of train images: ", dataset.__len__())
# print("Shape of image: ", dataset[0][0].shape)
# print("Batch size: ", config['batch_size'])
# print("Train batches: ", train_loader.__len__())
# print("Val batches: ", val_loader.__len__())

In [9]:
# model = Network().to(device)
# summary(model, (3, 256, 256))

#  Network 

In [10]:
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 7000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x, return_feats = False):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        feats = torch.flatten(x, 1)

        out = self.fc(feats)

        if return_feats:
            return feats
        else:
            return out
             
        
class Ensemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(Ensemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        
    def forward(self, x):
        self.x1 = self.modelA(x)
        self.x2 = self.modelB(x)
        x = torch.cat((self.x1, self.x2), dim=1)
        return x

In [11]:
#ResNet50
model_resnet = ResNet(Bottleneck, [3,4,6,3], num_classes=39)
model_resnet.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model_resnet, (3, 224, 224))
checkpoint = torch.load(config['checkpointPath'][1]) #,map_location=torch.device('cpu')
model_resnet.load_state_dict(checkpoint["model_state_dict"])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

<All keys matched successfully>

In [12]:
#ConvNext
class CNN(torch.nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        self.conv_layers = torch.nn.Sequential(
            # conv1
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.MaxPool2d(2),
            # conv2
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(64),
            torch.nn.MaxPool2d(2),
            # conv3
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),
            torch.nn.MaxPool2d(2),
            # conv4
            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(256),
            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(256),
            torch.nn.MaxPool2d(2),
        )

        self.dense_layers = torch.nn.Sequential(
            torch.nn.Dropout(0.4),
            torch.nn.Linear(50176, 1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(1024, K),
        )

    def forward(self, X):
        out = self.conv_layers(X)

        # Flatten
        out = out.view(-1, 50176)

        # Fully connected
        out = self.dense_layers(out)

        return out


In [13]:
#ConvNext
targets_size =39
model_convNext = CNN(targets_size)
model_convNext.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model_convNext, (3, 224, 224))
checkpoint = torch.load(config['checkpointPath'][0]) #,map_location=torch.device('cpu')
model_convNext.load_state_dict(checkpoint["model_state_dict"])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
              ReLU-2         [-1, 32, 224, 224]               0
       BatchNorm2d-3         [-1, 32, 224, 224]              64
            Conv2d-4         [-1, 32, 224, 224]           9,248
              ReLU-5         [-1, 32, 224, 224]               0
       BatchNorm2d-6         [-1, 32, 224, 224]              64
         MaxPool2d-7         [-1, 32, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          18,496
              ReLU-9         [-1, 64, 112, 112]               0
      BatchNorm2d-10         [-1, 64, 112, 112]             128
           Conv2d-11         [-1, 64, 112, 112]          36,928
             ReLU-12         [-1, 64, 112, 112]               0
      BatchNorm2d-13         [-1, 64, 112, 112]             128
        MaxPool2d-14           [-1, 64,

<All keys matched successfully>

In [14]:
#ENSEMBLE MODEL
model = Ensemble(model_resnet,model_convNext)
model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [15]:
checkpoint = torch.load('checkpointdiseasef.pth') #,map_location=torch.device('cpu')
model.load_state_dict(checkpoint["model_state_dict"])
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

# Setup everything for training

In [16]:
# criterion = torch.nn.CrossEntropyLoss() # TODO: What loss do you need for a multi class classification problem?
# optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9, weight_decay=1e-4)
# # checkpoint = torch.load(config['checkpointPath'])
# # model.load_state_dict(checkpoint["model_state_dict"])
# # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.8, patience=1)
# # scheduler = torch.optim.lr_scheduler.mulStepLR(optimizer, gamma=0.1, step_size=10)
# # scheduler= torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,30], gamma=0.1, last_epoch=- 1, verbose=False)
# cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 100, eta_min=0, last_epoch=-1)
# scheduler = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=5, after_scheduler=cosine_scheduler)
# # # device = torch.device("cuda" if cuda else "cpu")
# # model.cuda()
# # TODO: Implement a scheduler (Optional but Highly Recommended)
# # You can try ReduceLRonPlateau, StepLR, MultistepLR, CosineAnnealing, etc.
# scaler = torch.cuda.amp.GradScaler() # Good news. We have FP16 (Mixed precision training) implemented for you
# # It is useful only in the case of compatible GPUs such as T4/V100

# criterion = torch.nn.CrossEntropyLoss()  # this include softmax + cross entropy loss
# optimizer = torch.optim.Adam(model.parameters())

criterion = torch.nn.CrossEntropyLoss()
optimizer_conv = torch.optim.AdamW(model.parameters(), lr=0.001)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# checkpoint = torch.load(config['checkpointPath']) #,map_location=torch.device('cpu')
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer_conv.load_state_dict(checkpoint['optimizer_state_dict'])


# Let's train!

In [17]:
def train(model, criterion, optimizer, scheduler):

    
    model.train()
    batch_bar = tqdm(total=len(dataloaders['train']), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    running_loss = 0.0
    running_corrects = 0
    
    for i,(inputs, labels) in enumerate(dataloaders['train']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # if phase == 'train':
            loss.backward()
            optimizer.step()
                
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)


        scheduler.step()
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * running_corrects / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(running_loss / (i + 1))),
            running_corrects=running_corrects,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # scaler.scale(loss).backward() # This is a replacement for loss.backward()
        # scaler.step(optimizer) # This is a replacement for optimizer.step()
        # scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar
        # clear computation cache
        torch.cuda.empty_cache()
        # del images
        # del labels
        # del loss

    batch_bar.close() # You need this to close the tqdm bar

    # acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    # total_loss = float(total_loss / len(dataloader))
    epoch_loss = running_loss / data_size['train']
    epoch_acc = 100*running_corrects.double() / data_size['train']

    return epoch_acc, epoch_loss


In [18]:
def validate(model, criterion, optimizer, scheduler):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloaders['val']), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    # model.train()

    running_loss = 0.0
    running_corrects = 0
    
    for i,(inputs, labels) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            # if phase == 'train':
            loss.backward()
            optimizer.step()
                
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # if phase == 'train':
    # scheduler.step()
                
    # epoch_loss = running_loss / data_size['train']
    # epoch_acc = 100*running_corrects.double() / data_size['train']
            
            # print('{} Loss: {:.4f} Train Acc: {:.4f}'.format(
            #     phase, epoch_loss, epoch_acc))

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * running_corrects / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(running_loss / (i + 1))),
            running_corrects=running_corrects,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # scaler.scale(loss).backward() # This is a replacement for loss.backward()
        # scaler.step(optimizer) # This is a replacement for optimizer.step()
        # scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar
        # clear computation cache
        torch.cuda.empty_cache()
        # del images
        # del labels
        # del loss

    batch_bar.close() # You need this to close the tqdm bar

    # acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    # total_loss = float(total_loss / len(dataloader))
    epoch_loss = running_loss / data_size['val']
    epoch_acc = 100*running_corrects.double() / data_size['val']

    return epoch_acc, epoch_loss

In [19]:
import time
import copy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                batch_bar = tqdm(total=len(dataloaders['train']), dynamic_ncols=True, position=0, leave=False, desc='Train', ncols=5)
            else:
                model.eval()
                batch_bar = tqdm(total=len(dataloaders['val']), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)
                
            running_loss = 0.0
            running_corrects = 0
            
            for i,(inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / data_size[phase]
            epoch_acc = running_corrects.double() / data_size[phase]
            
            batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * running_corrects / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(running_loss / (i + 1))),
            running_corrects=running_corrects,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            
#             print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

            wandb.log({"train_loss":epoch_loss, 'train_Acc': epoch_acc})
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
#                 val_=epoch_acc
                best_acc = epoch_acc
                
                # best_model_wts = copy.deepcopy(model.state_dict())
                print("Saving model")
                torch.save({'model_state_dict':model.state_dict(),
                            'optimizer_state_dict':optimizer_conv.state_dict(),
                            #'scheduler_state_dict':scheduler.state_dict(),
                            'epoch_acc': epoch_acc, 
                            'epoch': epoch}, './checkpointdiseasef.pth')
                # best_valacc = val_acc
                wandb.save('checkpointdiseasef.pth')
                
#             print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(epoch_acc, val_loss))
            if phase=='val':
                wandb.log({'validation_Acc':epoch_acc, 
                   'validation_loss': epoch_loss})
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [20]:
# criterion = torch.nn.CrossEntropyLoss()
# optimizer_conv = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [21]:
gc.collect() # These commands help you when you face CUDA OOM error
torch.cuda.empty_cache()
# free_gpu_cache()  

# Wandb

In [22]:
wandb.login(key="0df4b2e8d09c7bfafcc61974b29dd9ab7ed479b6") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: sntivugu (ruffers). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc


True

In [24]:
# Create your wandb run
run = wandb.init(
    name = "Ensemble model resumed", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
#     id = '2350zm5y', ###Insert specific run id here if you want to resume a previous run
#     resume = True, ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "Crop_disease_midterm", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

# Experiments

In [25]:
# free_gpu_cache() 

model_resnet = train_model(model, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=15)

Epoch 0/14
----------


Train:   0%| | 0/13862 [24:07<?, ?it/s, acc=91.9131%, loss=1.0208, lr=0.0010, running_corrects=tensor(50964, device='cud

train Loss: 0.2552 Acc: 0.9191


Val:   0%| | 0/13862 [05:48<?, ?it/s, acc=93.2026%, loss=1.7541, lr=0.0010, running_corrects=tensor(51679, device='cuda:

val Loss: 0.4385 Acc: 0.9320
Saving model

Epoch 1/14
----------


...

Train:   0%| | 0/13862 [23:40<?, ?it/s, acc=98.1190%, loss=0.2444, lr=0.0000, running_corrects=tensor(54405, device='cud

train Loss: 0.0611 Acc: 0.9812


Val:   0%| | 0/13862 [05:31<?, ?it/s, acc=99.2750%, loss=0.2850, lr=0.0000, running_corrects=tensor(55046, device='cuda:

val Loss: 0.0712 Acc: 0.9927
Saving model



Training complete in 445m 2s
Best val Acc: 0.992750


In [ ]:
# best_valacc = 0.0

# for epoch in range(config['epochs']):

#     curr_lr = float(optimizer_conv.param_groups[0]['lr'])

#     train_acc, train_loss = train(model, criterion, optimizer_conv, exp_lr_scheduler)
#     # train_losses, validation_losses = batch_gd(
#     # model, criterion, train_loader, validation_loader, 5)
    
#     print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
#         epoch + 1,
#         config['epochs'],
#         train_acc,
#         train_loss,
#         curr_lr))
    
#     val_acc, val_loss = validate(model, criterion, optimizer_conv, exp_lr_scheduler)
    
#     print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

#     wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
#                'validation_loss': val_loss, "learning_Rate": curr_lr})
    
#     # If you are using a scheduler in your train function within your iteration loop, you may want to log
#     # your learning rate differently 

#     # #Save model in drive location if val_acc is better than best recorded val_acc
#     if val_acc >= best_valacc:
#       #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
#       print("Saving model")
#       torch.save({'model_state_dict':model.state_dict(),
#                   'optimizer_state_dict':optimizer_conv.state_dict(),
#                   #'scheduler_state_dict':scheduler.state_dict(),
#                   'val_acc': val_acc, 
#                   'epoch': epoch}, './drive/MyDrive/checkpointdiseasefv3.pth')
#       best_valacc = val_acc
#       wandb.save('checkpointdiseasefv2.pth')
#       # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()

# Classification Task: Testing

In [ ]:
# def test(model,dataloader):

#   model.eval()
#   batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Test')
#   test_results = []
  
#   for i, (images) in enumerate(dataloader):
#       # TODO: Finish predicting on the test set.
#       images = images.to(device)

#       with torch.inference_mode():
#         outputs = model(images)

#       outputs = torch.argmax(outputs, axis=1).detach().cpu().numpy().tolist()
#       test_results.extend(outputs)
      
#       batch_bar.update()
      
#   batch_bar.close()
#   return test_results

In [ ]:
# test_results = test(model, test_loader)